## 모델1

### 데이터

In [44]:
from sklearn.metrics import mean_absolute_error
from itertools import combinations
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer, MinMaxScaler,StandardScaler
from tensorflow.keras.layers import BatchNormalization, Dropout, Concatenate, Add
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
import tensorflow
from tensorflow_addons.optimizers import AdamW
from keras import backend as K
from sklearn.model_selection import train_test_split
import os
import time
from sklearn.model_selection import KFold
from tensorflow.keras.metrics import MeanAbsolutePercentageError

In [37]:
df = pd.read_csv('csv/Regression_data_preprocessing.csv')
df
target = 'Rings'
X = df.drop([target], axis=1)
y = df[target]

In [38]:
# train test val split
feature_cols = df.columns.tolist()
feature_cols.remove('Rings')
target_cols = ['Rings']
remove_list = ['Sex_I','Sex_F','Sex_M']
for col in remove_list:
    feature_cols.remove(col)
pipeline = Pipeline([('normalizer', Normalizer()),
                     ('scaler', StandardScaler())])

X_train, X_test = train_test_split(X, test_size= 0.2, random_state = 42)
y_train, y_test = train_test_split(y, test_size= 0.2, random_state = 42)
X_train, X_val = train_test_split(X_train, test_size = 0.25, random_state = 42)
y_train, y_val = train_test_split(y_train, test_size = 0.25, random_state = 42)

# pipeline을 통해 normalize와 standard scaler 적용
X_train[feature_cols] = pipeline.fit_transform(X_train[feature_cols])
X_test[feature_cols] = pipeline.transform(X_test[feature_cols])
X_val[feature_cols] = pipeline.transform(X_val[feature_cols])

### 모델 구현

In [45]:
def custom_opt2(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt

In [46]:
def accuracy(y_true, y_pred):
    return 1 - tf.abs((y_true - y_pred) / y_true)

In [48]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Concatenate, Add, BatchNormalization, Activation, Input
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [52]:
input_layer = tf.keras.layers.Input(shape=(X_train.shape[1],))
hidden1a = Dense(128, activation='relu')(input_layer)
hidden1b = Dense(128, activation='relu')(input_layer)
hidden1 = Concatenate()([hidden1a, hidden1b])
dropout1 = Dropout(0.2)(hidden1)

hidden2a = Dense(64, activation='relu')(dropout1)
hidden2b = Dense(64, activation='relu')(dropout1)
hidden2 = Add()([hidden2a, hidden2b])
dropout2 = Dropout(0.2)(hidden2)

hidden3a = Dense(32, activation='relu')(dropout2)
hidden3b = Dense(32, activation='relu')(dropout2)
hidden3 = Concatenate()([hidden3a, hidden3b])
dropout3 = Dropout(0.2)(hidden3)

output_layer = Dense(1)(dropout3)

model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

batch_lst = [8, 32, 64, 128, 256]
best_accuracy = 0.0
best_hyperparams = {}

for i in [0.01,0.006,0.003,0.001,0.0001]:
    for batch in batch_lst:
                
        # 모델 컴파일
        model.compile(loss='mse', optimizer=custom_opt2(i), metrics=[accuracy])

        # early stopping 구현 - 커스텀 정확도 기준
        early_stopping = EarlyStopping(monitor='val_accuracy', patience=30)
        start_time = time.time()
        model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping],verbose = 0)
        end_time = time.time()
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=2)

        if val_acc > best_accuracy:
                best_accuracy = val_acc
                best_hyperparams = {'batch_size': batch, 'learning_rate': i}

        
        

27/27 - 0s - loss: 4.7266 - accuracy: 0.8281 - 55ms/epoch - 2ms/step
27/27 - 0s - loss: 4.8570 - accuracy: 0.8248 - 55ms/epoch - 2ms/step
27/27 - 0s - loss: 4.7976 - accuracy: 0.8309 - 73ms/epoch - 3ms/step
27/27 - 0s - loss: 4.9594 - accuracy: 0.8262 - 47ms/epoch - 2ms/step
27/27 - 0s - loss: 4.6892 - accuracy: 0.8284 - 52ms/epoch - 2ms/step
27/27 - 0s - loss: 4.8547 - accuracy: 0.8383 - 46ms/epoch - 2ms/step
27/27 - 0s - loss: 4.8272 - accuracy: 0.8390 - 45ms/epoch - 2ms/step
27/27 - 0s - loss: 4.8336 - accuracy: 0.8320 - 50ms/epoch - 2ms/step
27/27 - 0s - loss: 4.8142 - accuracy: 0.8436 - 45ms/epoch - 2ms/step
27/27 - 0s - loss: 4.8913 - accuracy: 0.8432 - 63ms/epoch - 2ms/step
27/27 - 0s - loss: 5.1846 - accuracy: 0.8373 - 56ms/epoch - 2ms/step
27/27 - 0s - loss: 5.1768 - accuracy: 0.8366 - 54ms/epoch - 2ms/step
27/27 - 0s - loss: 4.9422 - accuracy: 0.8367 - 53ms/epoch - 2ms/step
27/27 - 0s - loss: 5.0476 - accuracy: 0.8397 - 45ms/epoch - 2ms/step
27/27 - 0s - loss: 5.3087 - accura

KeyboardInterrupt: 

In [ ]:
## 오르지 않는 것으로 확인 배제

In [ ]:
print(best_accuracy, best_hyperparams)

In [62]:
input_layer = tf.keras.layers.Input(shape=(X_train.shape[1],))
hidden1a = Dense(128, kernel_regularizer=l2(0.001))(input_layer)
hidden1a = BatchNormalization()(hidden1a)
hidden1a = Activation('relu')(hidden1a)

hidden1b = Dense(128, kernel_regularizer=l2(0.001))(input_layer)
hidden1b = BatchNormalization()(hidden1b)
hidden1b = Activation('relu')(hidden1b)

hidden1c = Dense(128, kernel_regularizer=l2(0.001))(input_layer)
hidden1c = BatchNormalization()(hidden1c)
hidden1c = Activation('relu')(hidden1c)

hidden1d = Dense(128, kernel_regularizer=l2(0.001))(input_layer)
hidden1d = BatchNormalization()(hidden1d)
hidden1d = Activation('relu')(hidden1d)

hidden1 = Concatenate()([hidden1a, hidden1b, hidden1c, hidden1d])
dropout1 = Dropout(0.2)(hidden1)

hidden2a = Dense(64, kernel_regularizer=l2(0.001))(dropout1)
hidden2a = BatchNormalization()(hidden2a)
hidden2a = Activation('relu')(hidden2a)

hidden2b = Dense(64, kernel_regularizer=l2(0.001))(dropout1)
hidden2b = BatchNormalization()(hidden2b)
hidden2b = Activation('relu')(hidden2b)

hidden2c = Dense(64, kernel_regularizer=l2(0.001))(dropout1)
hidden2c = BatchNormalization()(hidden2c)
hidden2c = Activation('relu')(hidden2c)

hidden2d = Dense(64, kernel_regularizer=l2(0.001))(dropout1)
hidden2d = BatchNormalization()(hidden2d)
hidden2d = Activation('relu')(hidden2d)

hidden2 = Add()([hidden2a, hidden2b, hidden2c, hidden2d])
dropout2 = Dropout(0.2)(hidden2)

output_layer = Dense(1)(dropout2)

model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
batch_lst = [8, 32, 64, 128, 256]
best_accuracy = 0.0
best_hyperparams = {}

for i in [0.01,0.006,0.003,0.001,0.0005]:
    for batch in batch_lst:
                
        # 모델 컴파일
        model.compile(loss='mse', optimizer=custom_opt2(i), metrics=[accuracy])

        # early stopping 구현 - 커스텀 정확도 기준
        early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
        start_time = time.time()
        model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_test, y_test), callbacks=[early_stopping],verbose = 0)
        end_time = time.time()
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=2)

        if val_acc > best_accuracy:
                best_accuracy = val_acc
                best_hyperparams = {'batch_size': batch, 'learning_rate': i}
        
print(best_accuracy, best_hyperparams)

27/27 - 0s - loss: 5.8677 - accuracy: 0.7982 - 161ms/epoch - 6ms/step
27/27 - 0s - loss: 4.7515 - accuracy: 0.8316 - 162ms/epoch - 6ms/step
27/27 - 0s - loss: 4.6510 - accuracy: 0.8344 - 152ms/epoch - 6ms/step
27/27 - 0s - loss: 4.5211 - accuracy: 0.8408 - 144ms/epoch - 5ms/step
27/27 - 0s - loss: 4.8068 - accuracy: 0.8417 - 127ms/epoch - 5ms/step
27/27 - 0s - loss: 4.6618 - accuracy: 0.8430 - 131ms/epoch - 5ms/step
27/27 - 0s - loss: 4.6025 - accuracy: 0.8499 - 143ms/epoch - 5ms/step
27/27 - 0s - loss: 4.5071 - accuracy: 0.8506 - 145ms/epoch - 5ms/step
27/27 - 0s - loss: 4.4481 - accuracy: 0.8435 - 148ms/epoch - 5ms/step
27/27 - 0s - loss: 4.4161 - accuracy: 0.8443 - 147ms/epoch - 5ms/step
27/27 - 0s - loss: 4.5057 - accuracy: 0.8449 - 139ms/epoch - 5ms/step
27/27 - 0s - loss: 4.3545 - accuracy: 0.8499 - 140ms/epoch - 5ms/step
27/27 - 0s - loss: 4.3542 - accuracy: 0.8500 - 138ms/epoch - 5ms/step
27/27 - 0s - loss: 4.4735 - accuracy: 0.8407 - 141ms/epoch - 5ms/step
27/27 - 0s - loss: 4

In [61]:
input_layer = tf.keras.layers.Input(shape=(X_train.shape[1],))
hidden1a = Dense(10, kernel_regularizer=l2(0.001))(input_layer)
hidden1a = BatchNormalization()(hidden1a)
hidden1a = Activation('relu')(hidden1a)

hidden1b = Dense(10, kernel_regularizer=l2(0.001))(input_layer)
hidden1b = BatchNormalization()(hidden1b)
hidden1b = Activation('relu')(hidden1b)

hidden1c = Dense(10, kernel_regularizer=l2(0.001))(input_layer)
hidden1c = BatchNormalization()(hidden1c)
hidden1c = Activation('relu')(hidden1c)

hidden1d = Dense(10, kernel_regularizer=l2(0.001))(input_layer)
hidden1d = BatchNormalization()(hidden1d)
hidden1d = Activation('relu')(hidden1d)

hidden1 = Concatenate()([hidden1a, hidden1b, hidden1c, hidden1d])
dropout1 = Dropout(0.2)(hidden1)

hidden2a = Dense(5, kernel_regularizer=l2(0.001))(dropout1)
hidden2a = BatchNormalization()(hidden2a)
hidden2a = Activation('relu')(hidden2a)

hidden2b = Dense(5, kernel_regularizer=l2(0.001))(dropout1)
hidden2b = BatchNormalization()(hidden2b)
hidden2b = Activation('relu')(hidden2b)

hidden2c = Dense(5, kernel_regularizer=l2(0.001))(dropout1)
hidden2c = BatchNormalization()(hidden2c)
hidden2c = Activation('relu')(hidden2c)

hidden2d = Dense(5, kernel_regularizer=l2(0.001))(dropout1)
hidden2d = BatchNormalization()(hidden2d)
hidden2d = Activation('relu')(hidden2d)

hidden2 = Add()([hidden2a, hidden2b, hidden2c, hidden2d])
dropout2 = Dropout(0.2)(hidden2)

output_layer = Dense(1)(dropout2)

model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
batch_lst = [10, 40, 80, 160, 320]
best_accuracy = 0.0
best_hyperparams = {}

for i in [0.01,0.006,0.003,0.001,0.0005]:
    for batch in batch_lst:
                
        # 모델 컴파일
        model.compile(loss='mse', optimizer=custom_opt2(i), metrics=[accuracy])

        # early stopping 구현 - 커스텀 정확도 기준
        early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
        start_time = time.time()
        model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_test, y_test), callbacks=[early_stopping],verbose = 0)
        end_time = time.time()
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=2)

        if val_acc > best_accuracy:
                best_accuracy = val_acc
                best_hyperparams = {'batch_size': batch, 'learning_rate': i}
        
print(best_accuracy, best_hyperparams)

27/27 - 0s - loss: 4.3696 - accuracy: 0.8534 - 56ms/epoch - 2ms/step
27/27 - 0s - loss: 4.5673 - accuracy: 0.8339 - 56ms/epoch - 2ms/step
27/27 - 0s - loss: 4.5300 - accuracy: 0.8354 - 54ms/epoch - 2ms/step
27/27 - 0s - loss: 4.5354 - accuracy: 0.8308 - 59ms/epoch - 2ms/step
27/27 - 0s - loss: 4.5213 - accuracy: 0.8350 - 54ms/epoch - 2ms/step
27/27 - 0s - loss: 4.4513 - accuracy: 0.8497 - 55ms/epoch - 2ms/step
27/27 - 0s - loss: 4.4535 - accuracy: 0.8396 - 58ms/epoch - 2ms/step
27/27 - 0s - loss: 4.3794 - accuracy: 0.8446 - 57ms/epoch - 2ms/step
27/27 - 0s - loss: 4.4008 - accuracy: 0.8393 - 55ms/epoch - 2ms/step
27/27 - 0s - loss: 4.5211 - accuracy: 0.8351 - 55ms/epoch - 2ms/step
27/27 - 0s - loss: 4.4714 - accuracy: 0.8430 - 57ms/epoch - 2ms/step
27/27 - 0s - loss: 4.3982 - accuracy: 0.8385 - 55ms/epoch - 2ms/step
27/27 - 0s - loss: 4.3816 - accuracy: 0.8388 - 54ms/epoch - 2ms/step
27/27 - 0s - loss: 4.3546 - accuracy: 0.8408 - 56ms/epoch - 2ms/step
27/27 - 0s - loss: 4.4050 - accura

In [60]:
input_layer = tf.keras.layers.Input(shape=(X_train.shape[1],))
# 모델 구성
model = Sequential()
model.add(Dense(256, input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(512, kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(1024, kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(512, kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(256, kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(1))

batch_lst = [8, 32, 64, 128, 256]
best_accuracy = 0.0
best_hyperparams = {}

for i in [0.01,0.006,0.003,0.001,0.0005]:
    for batch in batch_lst:
                
        # 모델 컴파일
        model.compile(loss='mse', optimizer=custom_opt2(i), metrics=[accuracy])

        # early stopping 구현 - 커스텀 정확도 기준
        early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
        start_time = time.time()
        model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_test, y_test), callbacks=[early_stopping],verbose = 0)
        end_time = time.time()
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=2)

        if val_acc > best_accuracy:
                best_accuracy = val_acc
                best_hyperparams = {'batch_size': batch, 'learning_rate': i}
        
print(best_accuracy, best_hyperparams)


27/27 - 0s - loss: 6.0189 - accuracy: 0.8288 - 430ms/epoch - 16ms/step
27/27 - 0s - loss: 6.6861 - accuracy: 0.8098 - 446ms/epoch - 17ms/step
27/27 - 0s - loss: 5.1090 - accuracy: 0.8540 - 417ms/epoch - 15ms/step
27/27 - 0s - loss: 5.4157 - accuracy: 0.8412 - 399ms/epoch - 15ms/step
27/27 - 0s - loss: 5.5428 - accuracy: 0.8359 - 441ms/epoch - 16ms/step
27/27 - 0s - loss: 6.1625 - accuracy: 0.8041 - 448ms/epoch - 17ms/step
27/27 - 0s - loss: 4.8455 - accuracy: 0.8361 - 414ms/epoch - 15ms/step
27/27 - 0s - loss: 4.8560 - accuracy: 0.8328 - 432ms/epoch - 16ms/step
27/27 - 0s - loss: 4.8151 - accuracy: 0.8256 - 422ms/epoch - 16ms/step
27/27 - 0s - loss: 4.8988 - accuracy: 0.8279 - 383ms/epoch - 14ms/step
27/27 - 0s - loss: 4.8433 - accuracy: 0.8316 - 402ms/epoch - 15ms/step
27/27 - 0s - loss: 4.7076 - accuracy: 0.8279 - 376ms/epoch - 14ms/step
27/27 - 0s - loss: 5.5292 - accuracy: 0.7913 - 383ms/epoch - 14ms/step
27/27 - 0s - loss: 5.0235 - accuracy: 0.8098 - 369ms/epoch - 14ms/step
27/27 

In [59]:
input_layer = tf.keras.layers.Input(shape=(X_train.shape[1],))
# 모델 구성
model = Sequential()
model.add(Dense(160, input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(320, kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(640, kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(320, kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(160, kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(1))

batch_lst = [10, 20, 40, 80, 160]
best_accuracy = 0.0
best_hyperparams = {}

for i in [0.01,0.006,0.003,0.001,0.0005]:
    for batch in batch_lst:
                
        # 모델 컴파일
        model.compile(loss='mse', optimizer=custom_opt2(i), metrics=[accuracy])

        # early stopping 구현 - 커스텀 정확도 기준
        early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
        start_time = time.time()
        model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_test, y_test), callbacks=[early_stopping],verbose = 0)
        end_time = time.time()
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=2)

        if val_acc > best_accuracy:
                best_accuracy = val_acc
                best_hyperparams = {'batch_size': batch, 'learning_rate': i}
        
print(best_accuracy, best_hyperparams)


27/27 - 0s - loss: 5.4978 - accuracy: 0.8238 - 209ms/epoch - 8ms/step
27/27 - 0s - loss: 7.7990 - accuracy: 0.7624 - 210ms/epoch - 8ms/step
27/27 - 0s - loss: 4.8383 - accuracy: 0.8378 - 203ms/epoch - 8ms/step
27/27 - 0s - loss: 5.2432 - accuracy: 0.8324 - 207ms/epoch - 8ms/step
27/27 - 0s - loss: 4.9110 - accuracy: 0.8250 - 205ms/epoch - 8ms/step
27/27 - 0s - loss: 5.3643 - accuracy: 0.8102 - 213ms/epoch - 8ms/step
27/27 - 0s - loss: 5.0833 - accuracy: 0.8178 - 208ms/epoch - 8ms/step
27/27 - 0s - loss: 4.9616 - accuracy: 0.8152 - 198ms/epoch - 7ms/step
27/27 - 0s - loss: 4.5266 - accuracy: 0.8431 - 198ms/epoch - 7ms/step
27/27 - 0s - loss: 4.6163 - accuracy: 0.8382 - 192ms/epoch - 7ms/step
27/27 - 0s - loss: 4.6403 - accuracy: 0.8378 - 183ms/epoch - 7ms/step
27/27 - 0s - loss: 4.7271 - accuracy: 0.8334 - 185ms/epoch - 7ms/step
27/27 - 0s - loss: 4.5664 - accuracy: 0.8362 - 209ms/epoch - 8ms/step
27/27 - 0s - loss: 4.5885 - accuracy: 0.8477 - 217ms/epoch - 8ms/step
27/27 - 0s - loss: 4

In [63]:
for i in [0.01,0.006,0.003,0.001,0.0005]:
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1))




    model.compile(optimizer=custom_opt2(i), loss='mean_squared_error', metrics=[accuracy])
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)

    # 모델 학습
    model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose = 0)

    # 검증 정확도 확인
    print(i)
    _, val_acc = model.evaluate(X_test, y_test)
    print("Validation Accuracy:", val_acc)

0.01
27/27 [==============================] - 0s 1ms/step - loss: 5.0585 - accuracy: 0.8491
Validation Accuracy: 0.849108099937439
0.006
27/27 [==============================] - 0s 1ms/step - loss: 4.5707 - accuracy: 0.8529
Validation Accuracy: 0.8528792262077332
0.003
27/27 [==============================] - 0s 1ms/step - loss: 4.4788 - accuracy: 0.8511
Validation Accuracy: 0.8510699272155762
0.001
27/27 [==============================] - 0s 1ms/step - loss: 4.5157 - accuracy: 0.8558
Validation Accuracy: 0.855845034122467
0.0005
27/27 [==============================] - 0s 1ms/step - loss: 4.6648 - accuracy: 0.8432
Validation Accuracy: 0.8431747555732727


### 최종

- 올리는 것에 실패 기존 모델 사용..

In [ ]:
for i in [0.01,0.006,0.003,0.001,0.0001]:
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1))




    model.compile(optimizer=custom_opt2(i), loss='mean_squared_error', metrics=[accuracy])
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)

    # 모델 학습
    model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose = 0)

    # 검증 정확도 확인
    print(i)
    _, val_acc = model.evaluate(X_test, y_test)
    print("Validation Accuracy:", val_acc)

In [28]:
# X_train = X_train.sort_index()
# y_train = y_train.sort_index()
# X_test = X_test.sort_index()
# y_test = y_test.sort_index()
# X_val = X_val.sort_index()
# y_val = y_val.sort_index()

In [31]:

# kf = KFold(n_splits=5, shuffle=True, random_state = 42)

# # 모델 생성
# model = Sequential()
# model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(1, activation='linear'))

# # 모델 컴파일
# model.compile(optimizer=custom_opt2(0.001), loss='mean_squared_error', metrics=[accuracy])

# # K-fold 교차 검증 실행
# fold = 1
# val_accuracies = []
# for train_index, val_index in kf.split(X_train):
#     print("Fold:", fold)
    
#     # 학습 데이터와 검증 데이터 분할
#     X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
#     y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    
#     # 모델 학습
#     model.fit(X_train_fold, y_train_fold, epochs=200, batch_size=32, verbose=0)
    
#     # 검증 정확도 평가
#     _, val_acc = model.evaluate(X_val_fold, y_val_fold)
#     val_accuracies.append(val_acc)
    
#     fold += 1

# # K-fold 교차 검증의 평균 검증 정확도 계산
# mean_val_accuracy = np.mean(val_accuracies)
# print("Mean Validation Accuracy:", mean_val_accuracy)